In [1]:
import pandas as pd
from collections import Counter, defaultdict
import arrow
import numpy as np
from scipy import stats

In [2]:
d = pd.read_csv('data/B2C_Rentalcover_24DEC2019.csv', 
                parse_dates=['FromDate','ToDate','CreatedOn', 'CreatedOnDate']) \
                    .drop_duplicates(['CustomerId', 'FromDate', 'ToDate', 'Type', 'CreatedOnDate'])

In [3]:
d_test = d[d["CustomerId"]==26384].sort_values('CreatedOn')

In [4]:
d_test.columns

Index(['CustomerId', 'BookingId', 'Reference', 'Type', 'FromDate', 'ToDate',
       'FromDayWeek', 'ToDayWeek', 'CreatedOn', 'CreatedOnDate', 'Cancelled',
       'DurationDays', 'UpfrontDays', 'QuoteWeek', 'QuoteDay', 'QuoteHour',
       'CurrencyId', 'Paid', 'Coupon', 'isCar', 'is4x4', 'isCamper',
       'isMinibus', 'isMotorHome', 'ToCountry', 'Lang', 'FirstName',
       'LastName', 'Email', 'ResCountry'],
      dtype='object')

In [15]:
fs_ = defaultdict(lambda: defaultdict(lambda: defaultdict()))

for i, row in enumerate(d_test.iterrows()):
    
    cid = row[1]['CustomerId']
    lab = 1 if row[1]['Type'] == 'Booking' else 0
    
    asis_features = ['FromDayWeek', 'ToDayWeek', 'Cancelled', 'DurationDays', 'UpfrontDays', 
                     'QuoteWeek', 'QuoteDay', 'QuoteHour', 'Paid', 'Coupon', 'isCar', 'is4x4', 'isCamper',
                       'isMinibus', 'isMotorHome', 'ToCountry', 'Lang', 'ResCountry']
    
    if i == 0:
        
        fs_[cid][i] = dict(prev_bookings=0, 
                           prev_quotes=0,
                           prev_cancel=0,
                           prev_bookings_this_country=0,
                           prev_quotes_this_country=0,
                           prev_bookings_this_week=0,
                           prev_quotes_this_week=0, 
                           prev_bookings_this_day=0,
                           prev_quotes_this_day=0,
                           prev_bookings_same_day=0,
                           prev_quotes_same_week=0, 
                           prev_quotes_same_trip=0,
                           conv=lab)
    else:
        prev_activities = d_test.iloc[:i]
        fs_[cid][i]['prev_bookings'] = Counter(prev_activities['Type'])['Booking']
        fs_[cid][i]['prev_quotes'] = Counter(prev_activities['Type'])['Quote']
        fs_[cid][i]['prev_cancel'] = prev_activities['Cancelled'].sum()
        
        fs_[cid][i]['prev_bookings_this_country'] = 0 if fs_[cid]['prev_bookings'] == 0 else \
                                                        sum([c == row[1]['ToCountry'] for c in prev_activities[prev_activities['Type'] == 'Booking']['ToCountry']])
        fs_[cid][i]['prev_quotes_this_country'] = 0 if fs_[cid]['prev_quotes'] == 0 else \
                                                        sum([c == row[1]['ToCountry'] for c in prev_activities[prev_activities['Type'] == 'Quote']['ToCountry']])
        fs_[cid][i]['prev_bookings_this_week'] = 0 if fs_[cid]['prev_bookings'] == 0 else \
                                                        sum([c == row[1]['QuoteWeek'] for c in prev_activities[prev_activities['Type'] == 'Booking']['QuoteWeek']])
        fs_[cid][i]['prev_quotes_this_week'] = 0 if fs_[cid]['prev_quotes'] == 0 else \
                                                        sum([c == row[1]['QuoteWeek'] for c in prev_activities[prev_activities['Type'] == 'Quote']['QuoteWeek']])
        fs_[cid][i]['prev_bookings_this_day'] = 0 if fs_[cid]['prev_bookings'] == 0 else \
                                                        sum([c == row[1]['QuoteDay'] for c in prev_activities[prev_activities['Type'] == 'Booking']['QuoteDay']])
        fs_[cid][i]['prev_quotes_this_day'] = 0 if fs_[cid]['prev_quotes'] == 0 else \
                                                        sum([c == row[1]['QuoteDay'] for c in prev_activities[prev_activities['Type'] == 'Quote']['QuoteDay']])
        fs_[cid][i]['prev_bookings_same_day'] = 0 if fs_[cid]['prev_bookings'] == 0 else \
                                                        sum([c == row[1]['CreatedOnDate'] for c in prev_activities[prev_activities['Type'] == 'Booking']['CreatedOnDate']])
        fs_[cid][i]['prev_quotes_same_day'] = 0 if fs_[cid]['prev_quotes'] == 0 else \
                                                        sum([c == row[1]['CreatedOnDate'] for c in prev_activities[prev_activities['Type'] == 'Quote']['CreatedOnDate']])
        fs_[cid][i]['prev_quotes_same_trip'] = 0 if fs_[cid]['prev_quotes'] == 0 else \
                                                        max(sum([arrow.get(c).shift(days=-2) <= arrow.get(row[1]['FromDate']) <= arrow.get(c).shift(days=+2) 
                                                                     for c in prev_activities[prev_activities['Type'] == 'Quote']['FromDate']]),
                                                           sum([arrow.get(c).shift(days=-2) <= arrow.get(row[1]['ToDate']) <= arrow.get(c).shift(days=+2) 
                                                                     for c in prev_activities[prev_activities['Type'] == 'Quote']['ToDate']]))
        fs_[cid][i]['conv'] = lab
        
    for ft in asis_features:
            fs_[cid][i][ft] = row[1][ft]
            
fs_[cid]['mult_currency'] = 1 if len(set(d_test['CurrencyId'])) > 1 else 0

In [16]:
fs_

defaultdict(<function __main__.<lambda>()>,
            {26384: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {0: {'prev_bookings': 0,
                           'prev_quotes': 0,
                           'prev_cancel': 0,
                           'prev_bookings_this_country': 0,
                           'prev_quotes_this_country': 0,
                           'prev_bookings_this_week': 0,
                           'prev_quotes_this_week': 0,
                           'prev_bookings_this_day': 0,
                           'prev_quotes_this_day': 0,
                           'prev_bookings_same_day': 0,
                           'prev_quotes_same_week': 0,
                           'prev_quotes_same_trip': 0,
                           'conv': 1,
                           'FromDayWeek': 'Tue',
                           'ToDayWeek': 'Tue',
                           'Cancelled': 0,
                           'DurationDays': 7,
     

In [130]:
Counter(['a','a','m']).keys()

dict_keys(['a', 'm'])

In [115]:
d_test

,CustomerId,BookingId,Reference,Type,FromDate,ToDate,FromDayWeek,ToDayWeek,CreatedOn,CreatedOnDate,...,is4x4,isCamper,isMinibus,isMotorHome,ToCountry,Lang,FirstName,LastName,Email,ResCountry
21309,26384,14579939,V662-W8E3-INS,Booking,2017-10-31 00:00:00,2017-11-07 00:00:00,Tue,Tue,2017-10-31 04:30:07+00:00,2017-10-31,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21308,26384,17561568,O6BQ-WRZ8-INS,Booking,2017-12-12 00:00:00,2017-12-19 00:00:00,Tue,Tue,2017-12-11 06:39:05+00:00,2017-12-11,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21305,26384,22919571,OH95-42ME-INS,Booking,2018-02-13 00:00:00,2018-02-20 00:00:00,Tue,Tue,2018-02-12 07:22:10+00:00,2018-02-12,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21314,26384,25061938,0579-X4R3-INS,Quote,2018-03-06 00:00:00,2018-03-13 00:00:00,Tue,Tue,2018-03-05 20:47:03+00:00,2018-03-05,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21313,26384,25062606,0BS1-8THJ-INS,Booking,2018-03-06 00:00:00,2018-03-13 00:00:00,Tue,Tue,2018-03-05 20:54:37+00:00,2018-03-05,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21315,26384,29269197,009I-6MVI-INS,Booking,2018-04-17 00:00:00,2018-04-24 00:00:00,Tue,Tue,2018-04-14 10:45:57+00:00,2018-04-14,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21316,26384,55910504,0P5N-04IZ-INS,Booking,2018-12-04 05:34:43,2018-12-11 05:34:43,Tue,Tue,2018-12-03 21:35:31+00:00,2018-12-03,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21304,26384,60698898,UWF5-375E-INS,Booking,2019-01-12 00:00:00,2019-01-22 00:00:00,Sat,Tue,2019-01-09 09:51:12+00:00,2019-01-09,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21307,26384,70250590,346J-54OF-INS,Booking,2019-03-08 00:00:00,2019-03-18 00:00:00,Fri,Mon,2019-03-08 05:22:53+00:00,2019-03-08,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU
21312,26384,83518852,4MU2-YH5R-INS,Booking,2019-05-14 00:00:00,2019-05-20 00:00:00,Tue,Mon,2019-05-14 09:55:20+00:00,2019-05-14,...,0,0,0,0,AU,en,bruce,walsham,bruce.walsham@live.com.au,AU


In [98]:
trip_ids = ['CustomerId', 'FromDate', 'ToDate', 'ToCountry']
gg = d_test.drop(['BookingId', 'Reference', 'CreatedOn'], axis=1).groupby(trip_ids) \
            .agg({'Type': lambda x: '->'.join(list(x)),
                  'FromDayWeek': lambda x: '->'.join(list(x)),
                  'ToDayWeek': lambda x: '->'.join(list(x)),
                  'Cancelled': lambda x: set(x).pop(),
                  'DurationDays': lambda x: list(x),
                  'UpfrontDays': lambda x: list(x)})

In [99]:
gg.columns

Index(['Type', 'FromDayWeek', 'ToDayWeek', 'Cancelled', 'DurationDays',
       'UpfrontDays'],
      dtype='object')

In [100]:
gg

Type  \
CustomerId FromDate            ToDate              ToCountry                   
26384      2017-10-31 00:00:00 2017-11-07 00:00:00 AU                Booking   
           2017-12-12 00:00:00 2017-12-19 00:00:00 AU                Booking   
           2018-02-13 00:00:00 2018-02-20 00:00:00 AU                Booking   
           2018-03-06 00:00:00 2018-03-13 00:00:00 AU         Booking->Quote   
           2018-04-17 00:00:00 2018-04-24 00:00:00 AU                Booking   
           2018-12-04 05:34:43 2018-12-11 05:34:43 AU                Booking   
           2019-01-12 00:00:00 2019-01-22 00:00:00 AU                Booking   
           2019-03-08 00:00:00 2019-03-18 00:00:00 AU                Booking   
           2019-05-14 00:00:00 2019-05-20 00:00:00 AU                Booking   
           2019-07-15 00:00:00 2019-07-22 00:00:00 AU                Booking   
           2019-08-05 00:00:00 2019-08-12 00:00:00 AU                Booking   
           2019-08-26 00:00:00 2019-09-02 00:00:00 AU                Booking   
           2019-09-17 00:00:00 2019-09-23 00:00:00 AU                Booking   
           2019-10-08 00:00:00 2019-10-14 00:00:00 AU                Booking   

                                                             FromDayWeek  \
CustomerId FromDate            ToDate              ToCountry               
26384      2017-10-31 00:00:00 2017-11-07 00:00:00 AU                Tue   
           2017-12-12 00:00:00 2017-12-19 00:00:00 AU                Tue   
           2018-02-13 00:00:00 2018-02-20 00:00:00 AU                Tue   
           2018-03-06 00:00:00 2018-03-13 00:00:00 AU           Tue->Tue   
           2018-04-17 00:00:00 2018-04-24 00:00:00 AU                Tue   
           2018-12-04 05:34:43 2018-12-11 05:34:43 AU                Tue   
           2019-01-12 00:00:00 2019-01-22 00:00:00 AU                Sat   
           2019-03-08 00:00:00 2019-03-18 00:00:00 AU                Fri   
           2019-05-14 00:00:00 2019-05-20 00:00:00 AU                Tue   
           2019-07-15 00:00:00 2019-07-22 00:00:00 AU                Mon   
           2019-08-05 00:00:00 2019-08-12 00:00:00 AU                Mon   
           2019-08-26 00:00:00 2019-09-02 00:00:00 AU                Mon   
           2019-09-17 00:00:00 2019-09-23 00:00:00 AU                Tue   
           2019-10-08 00:00:00 2019-10-14 00:00:00 AU                Tue   

                                                             ToDayWeek  \
CustomerId FromDate            ToDate              ToCountry             
26384      2017-10-31 00:00:00 2017-11-07 00:00:00 AU              Tue   
           2017-12-12 00:00:00 2017-12-19 00:00:00 AU              Tue   
           2018-02-13 00:00:00 2018-02-20 00:00:00 AU              Tue   
           2018-03-06 00:00:00 2018-03-13 00:00:00 AU         Tue->Tue   
           2018-04-17 00:00:00 2018-04-24 00:00:00 AU              Tue   
           2018-12-04 05:34:43 2018-12-11 05:34:43 AU              Tue   
           2019-01-12 00:00:00 2019-01-22 00:00:00 AU              Tue   
           2019-03-08 00:00:00 2019-03-18 00:00:00 AU              Mon   
           2019-05-14 00:00:00 2019-05-20 00:00:00 AU              Mon   
           2019-07-15 00:00:00 2019-07-22 00:00:00 AU              Mon   
           2019-08-05 00:00:00 2019-08-12 00:00:00 AU              Mon   
           2019-08-26 00:00:00 2019-09-02 00:00:00 AU              Mon   
           2019-09-17 00:00:00 2019-09-23 00:00:00 AU              Mon   
           2019-10-08 00:00:00 2019-10-14 00:00:00 AU              Mon   

                                                              Cancelled  \
CustomerId FromDate            ToDate              ToCountry              
26384      2017-10-31 00:00:00 2017-11-07 00:00:00 AU                 0   
           2017-12-12 00:00:00 2017-12-19 00:00:00 AU                 0   
           2018-02-13 00:00:00 2018-02-20 00:00:00 AU                 0   
           201

In [45]:
fts_ = defaultdict(lambda: defaultdict())
cid = gg.index[0][0]

for i, row in enumerate(gg.iterrows()):
    
    this_country = row[0][-1]
    this_week = row[1]['QuoteWeek'][0] if row[1]['QuoteWeek'][0] else row[1]['PaidWeek'][0]
    
    print('tihs week ', this_week)
    
    if i == 0:
        
        fts_[cid]['prev_cancellations'] = 0
        fts_[cid]['prev_bookings'] = 0
        fts_[cid]['prev_quotes'] = 0
        fts_[cid]['prev_bookings_this_country'] = 0
        fts_[cid]['prev_quotes_this_country'] = 0
        fts_[cid]['prev_bookings_this_week'] = 0
        fts_[cid]['prev_quotes_this_week'] = 0
        fts_[cid]['prev_bookings_this_day'] = 0
        fts_[cid]['prev_quotes_this_day'] = 0
        
    else:
        
        prev_activities = gg.iloc[:i]
        print('previous activities:')
        print(prev_activities)
        
        fts_[cid]['prev_cancellations'] = prev_activities['Cancelled'].apply(lambda x: x[0]).sum()
        fts_[cid]['prev_bookings'] = prev_activities['Type'].apply(lambda x: 'Booking' in x).sum()
        fts_[cid]['prev_quotes'] = prev_activities['Type'].apply(lambda x: 'Quote' in x).sum()
        fts_[cid]['prev_bookings_this_country'] = 0 if fts_[cid]['prev_bookings'] == 0 else \
                                     sum([this_country == _[-1] for _ in prev_activities[prev_activities['Type'] == 'Booking'].index])
        fts_[cid]['prev_quotes_this_country'] = 0 if fts_[cid]['prev_bookings'] == 0 else \
                                     sum([this_country == _[-1] for _ in prev_activities[prev_activities['Type'] == 'Quote'].index])

        fts_[cid]['prev_bookings_this_week'] = 0 if fts_[cid]['prev_bookings'] == 0 else \
                    prev_actitities[prev_activities['Type'] == 'Booking']['PaidWeek'].apply(lambda x: this_week == x[0]).sum()
                                    
    print(fts_)
    if i == 2:
        break
    

tihs week  44
defaultdict(<function <lambda> at 0x123ee6a60>, {26384: defaultdict(None, {'prev_cancellations': 0, 'prev_bookings': 0, 'prev_quotes': 0, 'prev_bookings_this_country': 0, 'prev_quotes_this_country': 0, 'prev_bookings_this_week': 0, 'prev_quotes_this_week': 0, 'prev_bookings_this_day': 0, 'prev_quotes_this_day': 0})})
tihs week  50
previous activities:
                                             BookingId        Reference  \
CustomerId FromDate   ToDate     ToCountry                                
26384      2017-10-31 2017-11-07 AU         [14579939]  [V662-W8E3-INS]   

                                                 Type FromDayWeek ToDayWeek  \
CustomerId FromDate   ToDate     ToCountry                                    
26384      2017-10-31 2017-11-07 AU         [Booking]         [3]       [3]   

                                                              CreatedOn  \
CustomerId FromDate   ToDate     ToCountry                                
26384      2017-10

<ipython-input-45-eca5eb175408>:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_actitities[prev_activities['Type'] == 'Booking']['PaidWeek'].apply(lambda x: this_week == x[0]).sum()


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [11]:
def get_booking_features(df):
    
    feats_ = defaultdict(lambda: defaultdict())
    
    trip_ids = ['CustomerId', 'FromDate', 'ToDate', 'ToCountry']
    trip_attribs = ['Type', 'FromDayWeek', 'ToDayWeek','Cancelled', 'UpfrontDays',
                   'DurationDays', 'PaidWeek','PaidDay', 'PaidHour', 'CurrencyId','Paid',
                   'isCar', 'is4x4', 'isCamper', 'isMinibus', 'isMotorHome','Lang']
    
    gb = df.sort_values(['CustomerId', 'CreatedOn']).groupby(trip_ids)
    
    total_trips = len(gb)
    
    trip_list = gb.agg(list)
#     print(trip_list)
    
    cid = trip_list.index.get_level_values(0)[0]
    
    if total_trips > 1:
        
        last_trip = list(trip_list.iloc[-1].apply(lambda x: x.pop())[trip_attribs])
        
        feats_[cid]['last_trip'] = last_trip
        
        tp = trip_list.iloc[3]
        print(tp)
        
        prev_trip = list(trip_list.iloc[-3].apply(lambda x: x.pop())[trip_attribs])
        feats_[cid]['prev_trip'] = prev_trip
        print(feats_)
            
    
    return None
    
    
    

In [12]:
get_booking_features(d_test)

BookingId                                    [25061938, 25062606]
Reference                          [0579-X4R3-INS, 0BS1-8THJ-INS]
Type                                             [Quote, Booking]
FromDayWeek                                                [3, 3]
ToDayWeek                                                  [3, 3]
CreatedOn       [2018-03-05 20:47:03+00:00, 2018-03-05 20:54:3...
ConfirmedOn                      [NaT, 2018-03-05 20:58:06+00:00]
PaidOn                           [NaT, 2018-03-05 20:56:23+00:00]
Cancelled                                                  [0, 0]
DurationDays                                               [7, 7]
UpfrontDays                                                [0, 0]
QuoteWeek                                                [10, 10]
QuoteDay                                                   [2, 2]
QuoteHour                                             [20.0, nan]
PaidWeek                                              [nan, 10.0]
PaidDay   

In [ ]:
list(d.groupby())

In [93]:
d.columns

Index(['CustomerId', 'BookingId', 'Reference', 'Type', 'FromDate', 'ToDate',
       'FromDayWeek', 'ToDayWeek', 'CreatedOn', 'ConfirmedOn', 'PaidOn',
       'CancelledOn', 'DurationDays', 'UpfrontDays', 'QuoteWeek', 'QuoteDay',
       'QuoteHour', 'PaidWeek', 'PaidDay', 'PaidHour', 'CurrencyId', 'Paid',
       'CouponCode', 'isCar', 'is4x4', 'isCamper', 'isMinibus', 'isMotorHome',
       'ToCountry', 'Lang', 'FirstName', 'LastName', 'Email', 'ResCountry',
       'CreatedOnDay'],
      dtype='object')

In [96]:
cs = d[d["CustomerId"]==26384]

In [130]:
len(cs.groupby(['FromDate', 'ToDate', 'ToCountry']).agg(list).index)

14

In [3]:
utc_frm = 'YYYY-MM-DD hh:mm:ss UTC'

In [ ]:
cs

In [117]:
action_list = cs.groupby(['FromDate', 'ToDate', 'ToCountry']).agg(list).drop(['CustomerId', 'BookingId', 'Reference'], axis=1)

In [118]:
action_list

,,,Type,FromDayWeek,ToDayWeek,CreatedOn,ConfirmedOn,PaidOn,CancelledOn,DurationDays,UpfrontDays,QuoteWeek,...,is4x4,isCamper,isMinibus,isMotorHome,Lang,FirstName,LastName,Email,ResCountry,CreatedOnDay
FromDate,ToDate,ToCountry,,,,,,,,,,,,,,,,,,,,,
2017-10-31 00:00:00,2017-11-07 00:00:00,AU,[Booking],[3],[3],[2017-10-31 04:30:07+00:00],[2017-10-31 04:36:10+00:00],[2017-10-31 01:35:00+00:00],[NaT],[7],[0],[44],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2017-10-31]
2017-12-12 00:00:00,2017-12-19 00:00:00,AU,[Booking],[3],[3],[2017-12-11 06:39:05+00:00],[2017-12-11 06:44:01+00:00],[2017-12-11 06:42:33+00:00],[NaT],[7],[0],[50],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2017-12-11]
2018-02-13 00:00:00,2018-02-20 00:00:00,AU,[Booking],[3],[3],[2018-02-12 07:22:10+00:00],[2018-02-12 07:26:20+00:00],[2018-02-12 07:25:19+00:00],[NaT],[7],[0],[7],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2018-02-12]
2018-03-06 00:00:00,2018-03-13 00:00:00,AU,"[Quote, Booking]","[3, 3]","[3, 3]","[2018-03-05 20:47:03+00:00, 2018-03-05 20:54:3...","[NaT, 2018-03-05 20:58:06+00:00]","[NaT, 2018-03-05 20:56:23+00:00]","[NaT, NaT]","[7, 7]","[0, 0]","[10, 10]",...,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[en, en]","[bruce, bruce]","[walsham, walsham]","[bruce.walsham@live.com.au, bruce.walsham@live...","[AU, AU]","[2018-03-05, 2018-03-05]"
2018-04-17 00:00:00,2018-04-24 00:00:00,AU,[Booking],[3],[3],[2018-04-14 10:45:57+00:00],[2018-04-14 10:52:08+00:00],[2018-04-14 08:50:08+00:00],[NaT],[7],[2],[15],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2018-04-14]
2018-12-04 05:34:43,2018-12-11 05:34:43,AU,[Booking],[3],[3],[2018-12-03 21:35:31+00:00],[2018-12-03 22:53:29+00:00],[2018-12-03 22:53:29+00:00],[NaT],[7],[0],[49],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2018-12-03]
2019-01-12 00:00:00,2019-01-22 00:00:00,AU,[Booking],[7],[3],[2019-01-09 09:51:12+00:00],[2019-01-09 10:52:47+00:00],[2019-01-10 09:31:20+00:00],[NaT],[10],[2],[2],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2019-01-09]
2019-03-08 00:00:00,2019-03-18 00:00:00,AU,[Booking],[6],[2],[2019-03-08 05:22:53+00:00],[2019-03-08 06:29:22+00:00],[2019-03-08 06:29:22+00:00],[NaT],[10],[0],[10],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2019-03-08]
2019-05-14 00:00:00,2019-05-20 00:00:00,AU,[Booking],[3],[2],[2019-05-14 09:55:20+00:00],[2019-05-14 09:58:56+00:00],[2019-05-14 09:58:55+00:00],[NaT],[6],[0],[20],...,[0],[0],[0],[0],[en],[bruce],[walsham],[bruce.walsham@live.com.au],[AU],[2019-05-14]


In [119]:
dik = defaultdict()

def get_features(action_list):
    
    last_action = action_list.iloc[-1]
    if len(last_action['Type']) == 1:  # only booking or quote but not bith for this trip
        if last_action['Type'][0] == 'Booking':
            cid = last_action['CustomerId'][0]
            dik[]
    
        

SyntaxError: invalid syntax (<ipython-input-119-20ee9ff660be>, line 9)

In [5]:
cls = ['CustomerId', 'Type', 'LanguageId', 'CreatedOnDay', 'FromDayWeek', 'ToDayWeek', 'DurationDays', 'UpfrontDays',
       'QuoteWeek', 'QuoteDay', 'QuoteHour', 'PaidWeek', 'PaidDay', 'PaidHour',
       'CurrencyId', 'Paid', 'isCar', 'ToCountry']

In [25]:
d1 = d.sort_values('CreatedOn').loc[d['CustomerId']==1683514, cls] \
                    .fillna({'QuoteHour': 25, 'PaidWeek': 53, 'PaidDay': 8, 'PaidHour': 25})
d1['dt'] = d1['CreatedOnDay'].apply(lambda x: round((arrow.get(d1.iloc[-1]['CreatedOnDay']) - arrow.get(x)).days/365.0*12.0, 2))

In [66]:
d1

,CustomerId,Type,LanguageId,CreatedOnDay,FromDayWeek,ToDayWeek,DurationDays,UpfrontDays,QuoteWeek,QuoteDay,QuoteHour,PaidWeek,PaidDay,PaidHour,CurrencyId,Paid,isCar,ToCountry,dt
0,1683514,Booking,5,2018-09-09,7,5,12,12,36,1,25.0,36.0,1.0,16.0,3,128.51,0,IS,7.3
2,1683514,Quote,5,2018-09-09,7,5,12,12,36,1,16.0,53.0,8.0,25.0,3,133.71,0,IS,7.3
1,1683514,Booking,1,2019-04-19,7,7,14,91,16,6,25.0,16.0,6.0,20.0,3,99.81,1,PT,0.0


In [86]:
ee = d1.groupby(['FromDayWeek', 'ToDayWeek', 'DurationDays','ToCountry']).agg({'Type': lambda x: '->'.join(list(x)),
                                                                              'LanguageId': set,
                                                                              'QuoteHour': lambda x: stats.mode(list(x))})

In [85]:
ee

Type LanguageId  \
FromDayWeek ToDayWeek DurationDays ToCountry                              
7           5         12           IS         Booking->Quote        {5}   
            7         14           PT                Booking        {1}   

                                                  QuoteHour  
FromDayWeek ToDayWeek DurationDays ToCountry                 
7           5         12           IS         ([16.0], [1])  
            7         14           PT         ([25.0], [1])

In [61]:
fq = ['UpfrontDays', 'QuoteWeek', 'QuoteDay']

for i in range(len(ee)):
    
    rec = ee.iloc[i]
    print(ee.index[i])
    print('->'.join(rec.Type))
    print(np.mean(rec.UpfrontDays))
    print(np.mean(rec.QuoteWeek))
    
    print(set(rec.LanguageId).pop())

(7, 5, 12)
Booking->Quote
12.0
36.0
5
(7, 7, 14)
Booking
91.0
16.0
1


In [14]:
d[d['CustomerId']==1683514].sort_values('CreatedOn')

,CustomerId,BookingId,Reference,Type,LanguageId,FromDate,ToDate,FromDayWeek,ToDayWeek,CreatedOn,...,CurrencyId,Paid,CouponCode,isCar,ToCountry,FirstName,LastName,Email,ResCountry,CreatedOnDay
0,1683514,46565787,S30N-RX0X-INS,Booking,5,2018-09-22,2018-10-04,7,5,2018-09-09 15:43:09+00:00,...,3,128.51,bonvoyage-r6THk,0,IS,jorge,domingo duet,jordi.dd@gmail.com,AD,2018-09-09
2,1683514,46567395,9225-06HU-INS,Quote,5,2018-09-22,2018-10-04,7,5,2018-09-09 16:02:01+00:00,...,3,133.71,bonvoyage-yasce,0,IS,jorge,domingo duet,jordi.dd@gmail.com,AD,2018-09-09
1,1683514,78559302,E1AP-2HT3-INS,Booking,1,2019-07-20,2019-08-03,7,7,2019-04-19 20:17:13+00:00,...,3,99.81,bonvoyage-eIr5h,1,PT,jorge,domingo duet,jordi.dd@gmail.com,AD,2019-04-19


In [31]:
arrow.get('2018-09-09 15:43:09 UTC', utc_frm)

<Arrow [2018-09-09T15:43:09+00:00]>

In [45]:
d[d['CustomerId']==1683514].sort_values('CreatedOn')

,CustomerId,BookingId,Reference,Type,LanguageId,FromDate,ToDate,FromDayWeek,ToDayWeek,CreatedOn,...,CurrencyId,Paid,CouponCode,isCar,ToCountry,FirstName,LastName,Email,ResCountry,CreatedOnDay
0,1683514,46565787,S30N-RX0X-INS,Booking,5,2018-09-22,2018-10-04,7,5,2018-09-09 15:43:09+00:00,...,3,128.51,bonvoyage-r6THk,0,IS,jorge,domingo duet,jordi.dd@gmail.com,AD,2018-09-09
2,1683514,46567395,9225-06HU-INS,Quote,5,2018-09-22,2018-10-04,7,5,2018-09-09 16:02:01+00:00,...,3,133.71,bonvoyage-yasce,0,IS,jorge,domingo duet,jordi.dd@gmail.com,AD,2018-09-09
1,1683514,78559302,E1AP-2HT3-INS,Booking,1,2019-07-20,2019-08-03,7,7,2019-04-19 20:17:13+00:00,...,3,99.81,bonvoyage-eIr5h,1,PT,jorge,domingo duet,jordi.dd@gmail.com,AD,2019-04-19


In [22]:
Counter(d['FromDayWeek'].astype(str) + '->' + d['ToDayWeek'].astype(str)).most_common()

[('6->2', 2749),
 ('6->1', 2613),
 ('7->7', 2520),
 ('7->1', 1972),
 ('5->1', 1927),
 ('5->2', 1829),
 ('7->2', 1794),
 ('6->6', 1600),
 ('2->2', 1575),
 ('4->4', 1512),
 ('2->6', 1486),
 ('3->3', 1444),
 ('1->1', 1441),
 ('6->7', 1425),
 ('7->3', 1364),
 ('6->3', 1320),
 ('4->1', 1247),
 ('5->5', 1241),
 ('2->5', 1223),
 ('5->3', 1215),
 ('3->7', 1201),
 ('5->6', 1196),
 ('2->4', 1190),
 ('3->4', 1182),
 ('4->5', 1179),
 ('5->7', 1178),
 ('2->7', 1177),
 ('1->7', 1177),
 ('3->5', 1164),
 ('3->6', 1162),
 ('1->2', 1158),
 ('4->2', 1157),
 ('4->6', 1143),
 ('7->4', 1118),
 ('4->7', 1117),
 ('2->3', 1087),
 ('1->4', 1034),
 ('1->6', 1002),
 ('1->5', 955),
 ('6->4', 933),
 ('7->6', 931),
 ('4->3', 929),
 ('2->1', 927),
 ('3->2', 919),
 ('1->3', 914),
 ('7->5', 904),
 ('5->4', 866),
 ('6->5', 839),
 ('3->1', 812)]

In [2]:
class R:
    
    def __init__(self, mlfile, cfile):
        
        self.m = pd.read_csv(mlfile)
        self.c = pd.read_csv(cfile, dtype={'CustomerId': float})
        
    def fix(self):
        
        make_lower_cols = ['email', 'languagecode', 'first_name', 'countrycode', 'last_name']
        fix_date_cols = ['last_emailed', 'date_added', 'last_opened']
        useful_cols = ['email', 'first_name', 'last_name', 'date_added', 'last_emailed',
                        'last_clicked', 'last_opened', 'agentid', 'countrycode', 'customerid',
                        'kickbox_result', 'languagecode']
        
        
        _msk_first_name_avail = (self.m['first_name'].astype(str).str.lower().str.strip() != 'nan')
        _msk_first_name_other_avail = (self.m['firstname'].astype(str).str.lower().str.strip() != 'nan')
        
        _msk_take_other_name = _msk_first_name_other_avail & (~_msk_first_name_avail)
        
        self.m['first_name'] = self.m['first_name'].where(~_msk_take_other_name, self.m['firstname'])
        
        self.m[make_lower_cols] = self.m[make_lower_cols].applymap(lambda x: str(x).lower().strip())
        
        self.m[fix_date_cols] = self.m[fix_date_cols].applymap(lambda x: arrow.get(str(x), 'MM/DD/YYYY').format('YYYY-MM-DD') if str(x) != 'nan' else None)
        
        self.m = self.m[useful_cols]
        
        print(f'there are {self.m["email"].nunique():,} unique email addresses in the mailing list')
        
#         cids_incommon = len({_ for _ in (set(self.c['CustomerId']) & set(self.m['customerid'])) if _.istype(float)})
        
#         print(f'cids in common: {cids_incommon:,}')
        
#         cust_emails = {_ for _ in s(et(self.c["Email1"]) | set(self.c["Email2"])) if '@' in str(_)}
        
#         print(f'total customer emails: {len(cust_emails):,}')
#         emails_incommon = cust_emails & set(self.m['email'])
        
#         print(f'emails in common: {len(emails_incommon):,}')
        
        return self

In [3]:
r = R(mlfile='data/all_contacts_list_export.csv', 
      cfile='data/bq-results-20191212-153418-s8cqnfpum1ou.csv').fix()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


there are 827,099 unique email addresses in the mailing list


In [4]:
r.m.head()

,email,first_name,last_name,date_added,last_emailed,last_clicked,last_opened,agentid,countrycode,customerid,kickbox_result,languagecode
0,tamtzesiang@gmail.com,tze siang,tam,2019-10-30,2019-11-12,NaN,2019-11-12,28,sg,NaN,NaN,en
1,frauleinfink@yahoo.co.uk,iris,henderson,2016-12-12,2017-06-07,NaN,None,NaN,nan,144326.0,NaN,nan
2,122141419@qq.com,良俊,金,2018-06-12,2019-12-10,NaN,2019-12-11,28,cn,NaN,NaN,zh
3,mcdurbanaustralia@gmail.com,christine,durban,2018-12-04,2019-12-10,NaN,2019-02-19,NaN,au,NaN,deliverable,en
4,jan_willem.van_wijck@novartis.com,jan,van wijck,2018-12-04,2018-12-04,NaN,None,NaN,ch,NaN,risky,en


In [5]:
r.c.head()

,CustomerId,BookingId,Reference,Type,LanguageId,FromDate,ToDate,CreatedOn,ConfirmedOn,Days,...,Paid,CouponCode,isCar,ToCountry,FirstName,LastName,Email1,Email2,CRecCreatedOn,ResCountry
0,2417812.0,73786064,LYCZ-183Y-INS,Quote,25.0,2019-04-03 00:00:00,2019-04-04 00:00:00,2019-03-27 14:12:09 UTC,NaN,1,...,14.34,bonvoyage-Q7g9t,1,ES,oles,ryaboy,tallar65@gmail.com,NaN,2019-07-30 03:54:46 UTC,UA
1,2341313.0,90899689,DD19-1BX9-INS,Quote,1.0,2019-09-13 00:00:00,2019-09-16 00:00:00,2019-06-13 08:16:32 UTC,NaN,3,...,35.93,bonvoyage-TZUNv,1,DE,ihor,tkachenko,5fotostudio@ukr.net,NaN,2019-07-30 03:54:46 UTC,UA
2,2227967.0,66615576,N09Y-JFKN-INS,Quote,25.0,2019-03-02 00:00:00,2019-03-10 00:00:00,2019-02-15 11:45:09 UTC,NaN,8,...,67.96,bonvoyage-gmFQW,1,DE,valeriy,saranchov,vsaranchov@gmail.com,NaN,2019-07-30 03:54:46 UTC,UA
3,3417283.0,122222780,SURJ-292T-INS,Booking,25.0,2019-09-23 00:00:00,2019-09-27 00:00:00,2019-09-23 20:55:32 UTC,2019-09-23 20:57:35 UTC,4,...,31.70,bonvoyage-KPI3b,1,UA,anton,osipov,ant2002@ukr.net,NaN,2019-09-23 20:52:34 UTC,UA
4,2738144.0,87640484,5N64-Q330-INS,Booking,45.0,2019-06-01 00:00:00,2019-06-05 00:00:00,2019-05-31 07:15:58 UTC,2019-05-31 07:49:22 UTC,4,...,35.60,bonvoyage-JBt0t,1,ES,anna,melnykova,lelekaua@gmail.com,NaN,2019-07-30 03:54:46 UTC,UA


In [47]:
ww = r.c[['CustomerId','Reference','Type','CreatedOn','ToCountry','Days']] \
            .groupby(['CustomerId', 'Type']) \
            .nunique().query("ToCountry>3")

In [54]:
cust_who_boked = r.c[r.c['ConfirmedOn'].notnull()].drop_duplicates(['Reference', 'ToCountry', 'ConfirmedOn'])

In [59]:
cust_who_boked.groupby(['CustomerId']).agg(lambda x: list(x))

,BookingId,Reference,Type,LanguageId,FromDate,ToDate,CreatedOn,ConfirmedOn,Days,PaidWk,...,Paid,CouponCode,isCar,ToCountry,FirstName,LastName,Email1,Email2,CRecCreatedOn,ResCountry
CustomerId,,,,,,,,,,,,,,,,,,,,,
39.0,[48003292],[40DN-0UZ1-INS],[Booking],[1.0],[2018-09-24 00:00:00],[2018-09-27 00:00:00],[2018-09-24 02:07:17 UTC],[2018-09-24 02:42:16 UTC],[3],[39.0],...,[32.45],[bonvoyage-eTPRv],[1],[AU],[arlene],[mackay],[brokenhillhotel@hotmail.com],[nan],[2019-07-30 03:54:46 UTC],[AU]
215.0,[34009389],[9K2G-0IV4-INS],[Booking],[1.0],[2018-06-08 00:00:00],[2018-06-13 00:00:00],[2018-05-29 00:21:36 UTC],[2018-05-29 00:26:02 UTC],[5],[22.0],...,[49.71],[bonvoyage-PfLmi],[1],[AU],[dana],[dulak],[ddulak@redcrossblood.org.au],[nan],[2019-07-30 03:54:46 UTC],[AU]
276.0,"[4148885, 11577267, 116791828]","[1K1B-CD0V-INS, GI9K-T466-INS, 2N49-64BC-INS]","[Booking, Booking, Booking]","[nan, 1.0, 1.0]","[2017-03-26 10:30:00, 2017-09-26 10:45:00, 201...","[2017-04-02 16:45:00, 2017-10-02 12:30:00, 201...","[2017-02-28 07:35:12 UTC, 2017-09-13 01:08:28 ...","[2017-03-20 23:32:01 UTC, 2017-09-21 00:52:01 ...","[7, 6, 10]","[12.0, 38.0, 36.0]",...,"[78.27, 67.37, 94.52]","[nan, nan, nan]","[0, 0, 0]","[AU, AU, AU]","[greg, greg, greg]","[tate, tate, tate]","[greg@tateswim.com.au, greg@tateswim.com.au, g...","[nan, nan, nan]","[2019-07-30 03:54:46 UTC, 2019-07-30 03:54:46 ...","[AU, AU, AU]"
487.0,"[23562807, 3371410]","[80U4-36N0-INS, TD63-BDI2-INS]","[Booking, Booking]","[1.0, 1.0]","[2018-03-19 00:00:00, 2016-12-15 00:00:00]","[2018-03-25 00:00:00, 2016-12-18 00:00:00]","[2018-02-19 05:51:30 UTC, 2016-12-12 03:07:28 ...","[2018-02-19 22:38:08 UTC, 2016-12-12 22:16:11 ...","[6, 3]","[8.0, 50.0]",...,"[60.95, 35.76]","[bonvoyage-WDGHM, nan]","[1, 1]","[AU, AU]","[chris, chris]","[steiner, steiner]","[chris.steiner@vicash.com.au, chris.steiner@vi...","[nan, nan]","[2019-07-30 03:54:46 UTC, 2019-07-30 03:54:46 ...","[AU, AU]"
802.0,[5115163],[VG0Y-M79Y-INS],[Booking],[nan],[2017-06-08 10:00:00],[2017-06-11 10:30:00],[2017-05-27 02:27:00 UTC],[2017-06-03 00:02:01 UTC],[3],[22.0],...,[35.78],[nan],[0],[AU],[peter],[smith],[waitedtoolong@live.com],[nan],[2019-07-30 03:54:46 UTC],[AU]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790056.0,[148457605],[2KB3-3UU5-INS],[Booking],[1.0],[2020-04-09 00:00:00],[2020-04-13 00:00:00],[2019-12-10 21:55:28 UTC],[2019-12-10 23:00:14 UTC],[4],[50.0],...,[31.55],[bonvoyage-cha1j],[1],[ES],[aharon],[ron levy],[rl2501@gmail.com],[nan],[2019-12-10 21:30:31 UTC],[IL]
3790164.0,[148461622],[S0Y7-NY8V-INS],[Booking],[5.0],[2019-12-13 00:00:00],[2019-12-15 00:00:00],[2019-12-10 22:07:42 UTC],[2019-12-10 23:15:58 UTC],[2],[50.0],...,[19.86],[bonvoyage-615wo],[1],[ES],[luisa],[ruiz gil],[diosalu@hotmail.com],[nan],[2019-12-10 22:07:41 UTC],[ES]
3790263.0,[148470873],[2KBY-PWHW-INS],[Booking],[5.0],[2020-01-04 00:00:00],[2020-01-08 00:00:00],[2019-12-10 22:41:31 UTC],[2019-12-10 23:54:04 UTC],[4],[50.0],...,[28.58],[bonvoyage-vqzA7],[1],[ES],[antonio],[arias pires],[apires_02@hotmail.com],[nan],[2019-12-10 22:41:30 UTC],[ES]


In [49]:
ch = {i[0] for i in ww.index}

In [50]:
r.c[r.c['CustomerId'].isin(ch)].groupby('Email1').agg(lambda x: set(x))

,CustomerId,BookingId,Reference,Type,LanguageId,FromDate,ToDate,CreatedOn,ConfirmedOn,Days,...,CurrencyId,Paid,CouponCode,isCar,ToCountry,FirstName,LastName,Email2,CRecCreatedOn,ResCountry
Email1,,,,,,,,,,,,,,,,,,,,,
0125dandelion@gmail.com,{2422765.0},"{130523716, 87824103, 116342604, 116343480, 11...","{61H9-KW29-INS, 96H5-7IIX-INS, 4SE8-VD75-INS, ...","{Quote, Booking}","{8.0, 1.0, 7.0}","{2019-06-06 00:00:00, 2019-04-16 00:00:00, 201...","{2019-10-21 00:00:00, 2019-04-16 00:00:00, 201...","{2019-11-10 09:31:40 UTC, 2019-09-07 18:09:34 ...","{nan, 2019-09-07 18:15:02 UTC, 2019-03-28 17:3...","{9, 2, 4, 7}",...,{5},"{35.57, 70.39, 17.28, 50.81, 19.63, 22.2, 55.7...","{bonvoyage-OWaGQ, bonvoyage-2ACFs, bonvoyage-k...",{1},"{IT, NL, CH, TW, GB}",{chao-hsun},{chen},{nan},{2019-07-30 03:54:46 UTC},{GB}
0199095@mail.ru,{1228987.0},"{80187172, 115816361, 115817833, 126157324, 11...","{AG8O-1FD5-INS, 075G-T0O8-INS, 4T7F-501M-INS, ...","{Quote, Booking}",{25.0},"{2019-08-26 00:00:00, 2019-09-12 00:00:00, 201...","{2019-09-01 00:00:00, 2019-05-09 00:00:00, 201...","{2019-09-06 07:08:29 UTC, 2019-09-06 07:16:50 ...","{nan, 2019-04-28 12:23:36 UTC, 2019-09-06 07:2...","{8, 4, 5, 6}",...,{3},"{33.2, 39.95, 46.93, 51.6, 63.28}","{bonvoyage-pmvUG, bonvoyage-6wwrA, bonvoyage-k...",{1},"{FR, SI, IT, ES}",{alexander},{ivanov},{nan},{2019-07-30 03:54:46 UTC},{RU}
0527160@gmail.com,{2879409.0},"{103105986, 94037958, 104983463, 104985067, 10...","{UH56-A4M7-INS, M65M-CZ19-INS, Z016-5G89-INS, ...","{Quote, Booking}",{13.0},"{2019-08-16 00:00:00, 2019-07-06 00:00:00, 201...","{2019-09-04 00:00:00, 2019-08-29 00:00:00, 201...","{2019-11-15 09:01:41 UTC, 2019-08-04 09:38:32 ...","{nan, 2019-07-05 12:21:17 UTC, 2019-08-04 09:3...","{9, 10, 11, 13, 30}",...,"{65, 2, 3, 1, 5, 54}","{67.12, 101.36, 70.61, 104.12, 204.49, 77.53, ...","{bonvoyage-touQX, bonvoyage-cAEYm, bonvoyage-L...",{1},"{CH, DE, IT, US}",{chrlap},{refael},{nan},{2019-07-30 03:54:46 UTC},{IL}
13958133896@139.com,{285929.0},"{39785223, 4540298, 43038221, 42924431, 361870...","{25JF-072A-INS, RJVB-B87G-INS, NSJ4-WAA7-INS, ...","{Quote, Booking}","{8.0, 1.0, 7.0}","{2017-05-15 00:00:00, 2018-08-14 00:00:00, 201...","{2017-10-08 00:00:00, 2017-05-27 00:00:00, 201...","{2018-08-11 04:34:02 UTC, 2016-12-20 17:24:05 ...","{nan, 2017-09-05 11:04:03 UTC, 2017-01-10 09:4...","{2, 4, 6, 7, 8, 9, 12, 20}",...,"{1, 2, 5}","{nan, nan, nan, nan, nan, 0.0, 161.42, 37.5, 4...","{nan, bonvoyage-uShjb, bonvoyage-5al19, bonvoy...","{0, 1}","{CA, CN, AU, US}",{wang},{ji},{nan},{2019-07-30 03:54:46 UTC},{CN}
2296591@gmail.com,{2080014.0},"{118154368, 118156801, 116384003, 122827079, 1...","{31CK-3PQ3-INS, RY6F-TPB3-INS, 8E00-X6NW-INS, ...","{Quote, Booking}",{13.0},"{2019-09-10 00:00:00, 2019-09-16 00:00:00, 201...","{2019-09-10 00:00:00, 2019-08-29 00:00:00, 201...","{2019-09-07 20:12:36 UTC, 2019-01-10 09:22:43 ...","{nan, 2019-09-12 12:42:17 UTC, 2019-09-07 20:1...","{0, 1, 2, 4, 6, 10}",...,"{2, 5, 54}","{34.47, 37.37, 8.93, 9.57, 76.44, 19.58, 19.59...","{bonvoyage-AaCyl, bonvoyage-mertj, bonvoyage-K...",{1},"{PL, HU, DE, US}",{yitzchak meir},{sheinbach},{nan},{2019-07-30 03:54:46 UTC},{IL}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zhoro@zaharievi.com,{121910.0},"{113363974, 39585033, 12069394, 13402902, 5156...","{4HEQ-7HGN-INS, VXV5-T340-INS, 5B5Y-5Y97-INS, ...","{Quote, Booking}",{1.0},"{2017-09-22 00:00:00, 2019-08-31 00:00:00, 201...","{2019-10-14 00:00:00, 2019-07-28 00:00:00, 201...","{2019-08-30 19:11:42 UTC, 2019-02-27 17:36:37 ...","{nan, 2019-08-30 19:11:42 UTC, 2017-02-27 10:0...","{1, 2, 3, 4, 5, 6, 7, 9, 14}",...,"{3, 23}","{nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","{nan, bonvoyage-rb5Lx, bonvoyage-fhrdg}",{1},"{GR, FR, IT, ES, US}",{georgi},{zahariev},{zahariev@dynamosoftware.com},{2019-07-30 03:54:46 UTC},{BG}
zkudrna@gmail.com,{413898.0},"{12073792, 6382209, 66012290, 51430211, 665393...","{9FS6-88H4-INS, 5FP7-LTB9-INS, 945Z-36SW-